# Naive

$$
\mathcal{L} = 
$$

In [1]:
%load_ext autoreload
%autoreload 2

# Libraries

In [2]:
import pickle
from tqdm import tqdm
from itertools import product

In [3]:
from avalanche.benchmarks.classic import PermutedMNIST
from avalanche.models import SimpleMLP
from avalanche.training import Naive
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import (
    accuracy_metrics,
    bwt_metrics,
    forgetting_metrics,
)

from torch.nn import CrossEntropyLoss
from torch.optim import SGD
from torch import cuda, device

/Users/jesusoyanedelespinoza/.pyenv/versions/3.9.18/envs/tt2/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Custom Libraries

In [4]:
import sys

sys.path.append("../base_code/")

from base_code.constants import DATASETS_PATH, SAVED_METRICS_PATH
from base_code.seed import set_seed

# Constants

In [5]:
TORCH_DEVICE = device("cuda" if cuda.is_available() else "cpu")
N_EXPERIENCES = 10
EXPERIMENT_SEED = 1234
METRICS_SUBFOLDER = f"retraining_saturation"
METRICS_FILENAME = f"{METRICS_SUBFOLDER}/naive.pkl"
set_seed(EXPERIMENT_SEED)

# Dataset and definitions

## Dataset loading

We load state-of-the-art dataset Modified NIST

In [6]:
scenario = PermutedMNIST(N_EXPERIENCES, seed=EXPERIMENT_SEED, dataset_root=DATASETS_PATH)

## Scenario creation with train test streaming

In this point, we define our scenario considering a training where in every experience of it, a new class is presented. This is, first we train with a class $a$, the following experience we train with class $b$ ($a \neq b$)

In [7]:
train_stream = scenario.train_stream
test_stream = scenario.test_stream

## Evaluation metrics definition

In [8]:
eval_plugin = EvaluationPlugin(
    accuracy_metrics(experience=True, stream=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    
    loggers=[InteractiveLogger()]
)

## Model, Optimizer, Loss function and Strategy definition

* `model`: Multi Layer Perceptron
* `Optimizer`: Adam
* `Loss function`: Cross Entropy
* `Strategy`: Elastic Weight Consolidation

In [9]:
model = SimpleMLP(num_classes=scenario.n_classes, input_size=28 * 28, hidden_layers=2, hidden_size=100).to(TORCH_DEVICE)
optimizer = SGD(model.parameters(), lr=1e-3)
criterion = CrossEntropyLoss().to(TORCH_DEVICE)
strategy = Naive(
    model,
    optimizer,
    criterion,
    train_epochs=5,
    train_mb_size=128,
    evaluator=eval_plugin,
    eval_mb_size=128,
    device=TORCH_DEVICE,
)

# Training and evaluation

Revisar porque el entrenamiento se está comportando de forma rara

In [10]:
training_exps = product(range(5), train_stream)

for (_, experience) in tqdm(training_exps, total=5 * N_EXPERIENCES):
    strategy.train(experience)
    strategy.eval(test_stream)

  0%|          | 0/50 [00:00<?, ?it/s]

-- >> Start of training phase << --
100%|██████████| 469/469 [00:03<00:00, 128.25it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:03<00:00, 118.56it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:03<00:00, 132.03it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:03<00:00, 131.33it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:03<00:00, 131.86it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 135.89it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.7596
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 139.04it/s]
> Eval on experience 1 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp001 = 0.2051
-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 140.83it/s]
>

  2%|▏         | 1/50 [00:24<19:42, 24.13s/it]


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.2204
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = 0.0000
	StreamForgetting/eval_phase/test_stream = 0.0000
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.2354
-- >> Start of training phase << --
100%|██████████| 469/469 [00:03<00:00, 122.26it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:03<00:00, 133.56it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:03<00:00, 130.82it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:03<00:00, 129.91it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:03<00:00, 123.83it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 138.63it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.0193
	ExperienceForgetting/eval_phase/test_st

  4%|▍         | 2/50 [00:48<19:30, 24.38s/it]

...


> Eval on experience 9 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp009 = -0.0162
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp009 = 0.0162
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.8937
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.0104
	StreamForgetting/eval_phase/test_stream = 0.0104
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.9024
-- >> Start of training phase << --
100%|██████████| 469/469 [00:03<00:00, 130.78it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:03<00:00, 131.76it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:03<00:00, 131.17it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:03<00:00, 131.62it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:03<00:00, 131.34it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 140.63it/s]
> Eval on experience 0 (Ta

 98%|█████████▊| 49/50 [19:45<00:23, 23.63s/it]


> Eval on experience 9 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp009 = -0.0151
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp009 = 0.0151
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.8948
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.0109
	StreamForgetting/eval_phase/test_stream = 0.0109
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.9026
-- >> Start of training phase << --
100%|██████████| 469/469 [00:03<00:00, 131.20it/s]
Epoch 0 ended.
100%|██████████| 469/469 [00:03<00:00, 131.77it/s]
Epoch 1 ended.
100%|██████████| 469/469 [00:03<00:00, 130.99it/s]
Epoch 2 ended.
100%|██████████| 469/469 [00:03<00:00, 131.30it/s]
Epoch 3 ended.
100%|██████████| 469/469 [00:03<00:00, 131.88it/s]
Epoch 4 ended.
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 79/79 [00:00<00:00, 135.17it/s]
> Eval on experience 0 (Ta

100%|██████████| 50/50 [20:09<00:00, 24.19s/it]


> Eval on experience 9 (Task 0) from test stream ended.
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp009 = 0.9176
-- >> End of eval phase << --
	StreamBWT/eval_phase/test_stream = -0.0104
	StreamForgetting/eval_phase/test_stream = 0.0104
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.9039


# Get metrics

In [11]:
accuracies: dict[int, list[float]] = dict()
forgettings: dict[int, list[float]] = dict()
bwt: dict[int, list[float]] = dict()

for i in range(N_EXPERIENCES):
    filled_i = str(i).zfill(3)
    accuracies[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    forgettings[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceForgetting/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
    bwt[f"Task{i}"] = eval_plugin.get_all_metrics()[
        f"ExperienceBWT/eval_phase/test_stream/Task000/Exp{filled_i}"
    ][1]
        

accuracies["Overall"] = eval_plugin.get_all_metrics()[
    "Top1_Acc_Stream/eval_phase/test_stream/Task000"
][1]

# Store metrics

In [12]:
pickle.dump({
    "accuracies": accuracies,
    "forgettings": forgettings,
    "bwt": bwt,
}, open(SAVED_METRICS_PATH / METRICS_FILENAME, "wb"))